<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 11:10:19
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.44 L
Current:  1.43 C
-------------------
Today PnL: 48.72 K (0.34%)
Current PnL: -16.38 L (-11.02%)
CY Booked + Current PnL: -5.10 L (-3.43%)
-------------------
Total profit:  2.26 L
Total loss:  -18.65 L
-------------------
Total Booked + Current PnL: 21.88 L (17.98%)
Total Booked PnL: 38.26 L (31.44%)
Curr Year Booked PnL: 11.28 L (7.88%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.01 L (57.32%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 10.03%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.75,56.0,H-LC,10.85,172403.0,16226.0,9861.0,-0.65,10.39,5.72,16.71,22.0,1.65,1.24,29.64,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,104.48,57.0,M-SC,3.97,88137.0,-12640.0,12736.0,-0.58,-12.54,14.45,0.09,245.0,-0.99,0.64,15.79,OX40N,NTT,DURABLES
57,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,220818.0,10452.0,18328.0,0.12,4.97,8.30,13.68,37.0,0.57,1.59,22.31,XY25,NTT,REFINERIES
50,NATIONALUM,247.44,-43.66,64.0,H-MC,6.81,112810.0,12306.0,18332.0,-0.19,12.24,16.25,30.49,79.0,0.67,0.81,48.62,MH,ATH,METALS
42,ITC,452.00,-37.50,53.0,H-LC,2.51,201712.0,1574.0,19324.0,0.78,0.79,9.58,10.44,4.0,0.08,1.46,6.39,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.69,54.0,H-SC,17.08,200578.0,624.0,96598.0,-0.42,0.31,48.16,48.63,115.0,0.01,1.45,11.90,XR,NTT,MISC
30,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248087.0,356.0,74699.0,-0.20,0.14,30.11,30.30,92.0,0.00,1.79,13.45,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.68,53.0,H-MC,4.20,200745.0,-75.0,68856.0,-0.01,-0.04,34.30,34.25,80.0,-0.00,1.45,5.16,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-17.03,39.0,H-LC,6.38,298680.0,-17162.0,129836.0,-0.62,-5.43,43.47,35.67,5.0,-0.13,2.15,7.67,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.93,47.0,H-LC,7.84,219718.0,-32050.0,85251.0,0.13,-12.73,38.80,21.13,27.0,-0.38,1.59,18.19,X40,ATH,PAINTS
42,ITC,452.00,-37.50,53.0,H-LC,2.51,201712.0,1574.0,19324.0,0.78,0.79,9.58,10.44,4.0,0.08,1.46,6.39,X40,NTT,FMCG
44,JIOFIN,387.00,0.39,53.0,H-LC,13.23,272706.0,5341.0,60104.0,0.40,2.00,22.04,24.48,48.0,0.09,1.97,58.07,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.78,228880.0,-5264.0,53878.0,-0.48,-2.25,23.54,20.77,86.0,-0.10,1.65,4.51,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,0.87,72.0,H-LC,1.29,250178.0,4027.0,75178.0,2.52,1.64,30.05,32.18,16.0,0.05,1.80,38.27,X200,ATH,IT
42,ITC,452.0,-37.50,53.0,H-LC,2.51,201712.0,1574.0,19324.0,0.78,0.79,9.58,10.44,4.0,0.08,1.46,6.39,X40,NTT,FMCG
51,NESTLEIND,1377.0,-7.84,59.0,H-LC,4.24,283608.0,18182.0,38599.0,0.64,6.85,13.61,21.40,11.0,0.47,2.05,14.43,XY25,NTT,FMCG
57,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,220818.0,10452.0,18328.0,0.12,4.97,8.30,13.68,37.0,0.57,1.59,22.31,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-19.44,54.0,H-LC,5.78,214994.0,10494.0,30916.0,0.62,5.13,14.38,20.25,10.0,0.34,1.55,14.31,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,0.87,72.0,H-LC,1.29,250178.0,4027.0,75178.0,2.52,1.64,30.05,32.18,16.0,0.05,1.80,38.27,X200,ATH,IT
11,BANDHANBNK,400.00,-9.90,54.0,H-SC,5.13,225120.0,-53439.0,310891.0,2.26,-19.18,138.10,92.42,151.0,-0.17,1.62,30.10,XY24,NTT,BANKS
8,ATULAUTO,844.00,3954.05,65.0,M-SC,5.79,126331.0,-21037.0,72855.0,2.22,-14.28,57.67,35.16,236.0,-0.29,0.91,29.03,XY24,NTT,AUTO
47,LAOPALA,464.00,117.42,54.0,H-SC,2.35,99449.0,-31656.0,82901.0,2.19,-24.15,83.36,39.09,142.0,-0.38,0.72,28.08,AR,NTT,CERAMICS
16,BSOFT,836.99,-11.70,58.0,H-SC,15.00,99195.0,-41458.0,114233.0,2.10,-29.48,115.16,51.74,131.0,-0.36,0.72,13.12,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-4.34,47.0,H-SC,12.74,88971.0,-11992.0,106525.0,-1.18,-11.88,119.73,93.63,148.0,-0.11,0.64,32.52,SR,ATH,CHEMICALS
32,HINDZINC,730.22,21.48,59.0,M-LC,1.36,197555.0,-7521.0,120094.0,-1.15,-3.67,60.79,54.89,52.0,-0.06,1.43,19.54,X5K,ATH,METALS
69,SURYODAY,240.00,46.93,59.0,H-SC,2.49,134171.0,-44900.0,110624.0,-1.05,-25.07,82.45,36.71,135.0,-0.41,0.97,33.12,XR,NTT,BANKS
37,INDIAMART,4850.92,-52.03,41.0,H-SC,11.65,133894.0,10558.0,123209.0,-0.96,8.56,92.02,108.45,119.0,0.09,0.97,32.83,AR,ATH,MISC
27,GREENPANEL,537.00,235.92,64.0,M-SC,2.40,149962.0,-33116.0,112097.0,-0.86,-18.09,74.75,43.14,230.0,-0.30,1.08,41.06,XY24,NTT,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,104.48,57.0,M-SC,3.97,88137.0,-12640.0,12736.0,-0.58,-12.54,14.45,0.09,245.0,-0.99,0.64,15.79,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,96.03,56.0,H-SC,9.54,124200.0,-18279.0,37794.0,0.55,-12.83,30.43,13.70,163.0,-0.48,0.90,46.59,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.06,64.0,H-SC,1.40,226935.0,-42732.0,79064.0,1.07,-15.85,34.84,13.47,138.0,-0.54,1.64,14.78,XY24,NTT,PAINTS
19,CERA,9475.0,-19.33,50.0,H-SC,2.08,146866.0,-29037.0,71054.0,0.18,-16.51,48.38,23.89,149.0,-0.41,1.06,26.84,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-19.01,56.0,H-MC,7.63,109488.0,-22347.0,64576.0,1.40,-16.95,58.98,32.03,98.0,-0.35,0.79,22.64,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-25.15,59.0,H-LC,12.33,298347.0,-47609.0,122471.0,0.03,-13.76,41.05,21.64,1.0,-0.39,2.15,5.69,X40,ATH,IT
40,INFY,2275.00,-15.29,59.0,H-LC,8.40,329554.0,16666.0,155022.0,1.63,5.33,47.04,54.87,3.0,0.11,2.38,12.33,X40,BTT,IT
42,ITC,452.00,-37.50,53.0,H-LC,2.51,201712.0,1574.0,19324.0,0.78,0.79,9.58,10.44,4.0,0.08,1.46,6.39,X40,NTT,FMCG
83,VBL,671.64,-17.03,39.0,H-LC,6.38,298680.0,-17162.0,129836.0,-0.62,-5.43,43.47,35.67,5.0,-0.13,2.15,7.67,X40N,ATH,FMCG
1,ABB,7934.00,-37.63,67.0,H-LC,10.34,260496.0,-1123.0,120323.0,0.76,-0.43,46.19,45.57,7.0,-0.01,1.88,10.84,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,63.0,M-MC,6.95,227373.0,2411.0,165232.0,-0.53,1.07,72.67,74.53,192.0,0.01,1.64,33.67,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,13.77,65.0,H-MC,3.57,196500.0,31830.0,36136.0,1.28,19.33,18.39,41.28,89.0,0.88,1.42,28.85,X40N,ATH,AC
84,VOLTAS,1918.49,0.88,58.0,H-MC,2.48,211785.0,20043.0,75988.0,0.00,10.45,35.88,50.08,99.0,0.26,1.53,17.96,XY25,ATH,AC
18,CAMS,5211.76,0.16,66.0,H-SC,3.82,113792.0,11788.0,32135.0,1.25,11.56,28.24,43.06,122.0,0.37,0.82,30.65,X40N,ATH,MISC
30,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248087.0,356.0,74699.0,-0.20,0.14,30.11,30.30,92.0,0.00,1.79,13.45,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-43.66,64.0,H-MC,6.81,112810.0,12306.0,18332.0,-0.19,12.24,16.25,30.49,79.0,0.67,0.81,48.62,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,0.48,100508.0,9010.0,69743.0,-0.11,9.85,69.39,86.07,223.0,0.13,0.73,45.95,XR,NTT,DURABLES
37,INDIAMART,4850.92,-52.03,41.0,H-SC,11.65,133894.0,10558.0,123209.0,-0.96,8.56,92.02,108.45,119.0,0.09,0.97,32.83,AR,ATH,MISC
86,WIPRO,420.00,-9.26,60.0,M-LC,5.63,159390.0,8445.0,101005.0,1.15,5.59,63.37,72.51,53.0,0.08,1.15,11.95,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,6.14,184237.0,4429.0,107981.0,0.13,2.46,58.61,62.52,88.0,0.04,1.33,37.39,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1894.95,67.0,L-SC,3.05,54439.0,-82738.0,82829.0,0.29,-60.31,152.15,0.07,267.0,-1.00,0.39,35.39,OX40N,NTT,JEWELLERY
7,ASIANTILES,137.00,6694.44,52.0,L-SC,5.18,74540.0,-19270.0,95888.0,0.60,-20.54,128.64,81.67,269.0,-0.20,0.54,44.35,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,63.0,M-SC,2.64,82680.0,-19530.0,112916.0,2.02,-19.11,136.57,91.37,208.0,-0.17,0.60,34.22,XY25,NTT,FINANCE
22,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.32,84922.0,-28627.0,68676.0,-0.23,-25.21,80.87,35.27,268.0,-0.42,0.61,107.85,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,47.0,H-SC,12.74,88971.0,-11992.0,106525.0,-1.18,-11.88,119.73,93.63,148.0,-0.11,0.64,32.52,SR,ATH,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3954.05,65.0,M-SC,5.79,126331.0,-21037.0,72855.0,2.22,-14.28,57.67,35.16,236.0,-0.29,0.91,29.03,XY24,NTT,AUTO
81,VAIBHAVGBL,521.0,66.94,72.0,H-SC,7.21,144954.0,-37821.0,148897.0,1.51,-20.69,102.72,60.77,125.0,-0.25,1.05,29.83,XR,NTT,JEWELLERY
61,SAMMAANCAP,326.0,-171.59,63.0,M-SC,2.64,82680.0,-19530.0,112916.0,2.02,-19.11,136.57,91.37,208.0,-0.17,0.60,34.22,XY25,NTT,FINANCE
22,COFFEEDAY,80.0,-41.65,52.0,L-SC,42.32,84922.0,-28627.0,68676.0,-0.23,-25.21,80.87,35.27,268.0,-0.42,0.61,107.85,XR,NTT,HOTELS
75,TITAGARH,1548.0,1.18,69.0,H-SC,3.26,166075.0,-25940.0,104827.0,0.72,-13.51,63.12,41.08,158.0,-0.25,1.20,40.73,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,43.34
2,50,73.79


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.03
LC    33.00
MC    22.96
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.94
X40      15.25
X40N     13.01
XR       11.63
AR        8.66
XY25      8.16
OX40N     8.14
X200      1.80
MH        1.69
X5K       1.43
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.99
H-LC    26.35
H-MC    20.04
M-SC    13.50
M-LC     5.59
M-MC     2.62
L-SC     1.54
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.54,-12.08,73.19
FMCG,11.74,-4.12,40.55
FINANCE,9.64,-15.04,60.10
MISC,7.40,-12.12,74.48
BANKS,6.24,-15.28,75.93
PAINTS,5.81,-14.81,31.93
ELECTRICAL,5.68,-4.51,62.11
AC,3.66,4.42,27.95
INSURANCE,3.39,-6.81,43.45


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2832222.0,22
XR,1299508.0,14
AR,1297056.0,9
X40,735543.0,10
X40N,605778.0,10
OX40N,518220.0,10
XY25,392516.0,6
SR,243839.0,2
X5K,120094.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3393605.0,29
M-SC,1374784.0,17
H-LC,1222029.0,15
H-MC,1175362.0,15
M-LC,375655.0,4
M-MC,314146.0,2
L-SC,247393.0,3
L-MC,60419.0,1
L-LC,37051.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1020915.0      6
M-SC       XY24       793198.0      7
H-SC       AR         784125.0      5
           XR         751432.0      7
H-MC       XY24       489303.0      4
H-LC       X40        453128.0      5
           AR         352201.0      2
M-MC       XY24       314146.0      2
H-SC       X40N       284478.0      4
           OX40N      246658.0      4
           SR         243839.0      2
H-LC       X40N       224491.0      4
H-MC       X40        212739.0      4
           XY25       185622.0      2
L-SC       XR         164564.0      2
M-SC       AR         160730.0      2
M-LC       XY24       154556.0      2
M-SC       OX40N      124157.0      4
M-LC       X5K        120094.0      1
M-SC       XR         114107.0      2
           XY25       112916.0      1
H-MC       XR         107981.0      1
M-LC       XR         101005.0      1
H-MC       X40N        96809.0      2
L-SC       OX40N       82829.0      1
H-LC       X200        75178.0      1
M-SC       X40         69676.0      1
H-MC       OX40N       64576.0      1
H-SC       MH          62158.0      1
L-MC       XR          60419.0      1
H-LC       XY24        60104.0      1
           XY25        56927.0      2
L-LC       XY25        37051.0      1
H-MC       MH          18332.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
